<a href="https://colab.research.google.com/github/atelierkarin/fm-ca-predict/blob/master/FM_CA_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime

In [0]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
max_date = datetime.datetime.strptime('1970/1/1', '%Y/%m/%d')
max_diff = datetime.datetime.today() - max_date

def calc_dob_from_now(dob):
  diff = datetime.datetime.today() - datetime.datetime.strptime(dob, '%Y/%m/%d')
  return diff.total_seconds()

def reformat_internal_goals(app, gl):
  return gl / app if app > 0 else 0

def load_data():
  df_club = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/club.csv')
  df_competition = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/competition.csv')
  df_nationality = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/nationality.csv')
  df_players = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/players.csv')
  df_season = pd.read_csv('/content/drive/My Drive/datas/fm-ca-status/season.csv')

  df = df_players[:].copy()

  df['DOB_FromNow'] = df.apply(lambda row: calc_dob_from_now(row['DOB']), axis=1)
  df = df.drop(columns='DOB')

  df = pd.merge(df, df_nationality, left_on='Nationality', right_on='Nation Name', how='left').drop(columns='Nation Name').drop(columns='Nationality')
  df = pd.merge(df, df_club, left_on='Club', right_on='Club Name', how='left').drop(columns='Club Name').drop(columns='Club')
  df = pd.merge(df, df_competition, left_on='League', right_on='Competition Name', how='left').drop(columns='League').drop(columns='Competition Name')

  df_ly = df_season[:].copy()
  df_l2y = df_season[:].copy()
  df_l3y = df_season[:].copy()

  df_ly.columns = 'LY ' + df_ly.columns.values
  df_l2y.columns = 'L2Y ' + df_l2y.columns.values
  df_l3y.columns = 'L3Y ' + df_l3y.columns.values

  df_ly_club = df_club[:].copy()
  df_l2y_club = df_club[:].copy()
  df_l3y_club = df_club[:].copy()
  df_ly_competition = df_competition[:].copy()
  df_l2y_competition = df_competition[:].copy()
  df_l3y_competition = df_competition[:].copy()

  df_ly_club.columns = 'LY ' + df_ly_club.columns.values
  df_l2y_club.columns = 'L2Y ' + df_l2y_club.columns.values
  df_l3y_club.columns = 'L3Y ' + df_l3y_club.columns.values
  df_ly_competition.columns = 'LY ' + df_ly_competition.columns.values
  df_l2y_competition.columns = 'L2Y ' + df_l2y_competition.columns.values
  df_l3y_competition.columns = 'L3Y ' + df_l3y_competition.columns.values

  df = pd.merge(df, df_ly, left_on=['LY Year','LY Club','LY League'], right_on=['LY Season','LY Team','LY League'], how='left').drop(columns=['LY Year','LY Season','LY Team'])
  df = pd.merge(df, df_ly_club, left_on='LY Club', right_on='LY Club Name', how='left').drop(columns='LY Club Name').drop(columns='LY Club')
  df = pd.merge(df, df_ly_competition, left_on='LY League', right_on='LY Competition Name', how='left').drop(columns='LY Competition Name').drop(columns='LY League')

  df = pd.merge(df, df_l2y, left_on=['L2Y Year','L2Y Club','L2Y League'], right_on=['L2Y Season','L2Y Team','L2Y League'], how='left').drop(columns=['L2Y Year','L2Y Season','L2Y Team'])
  df = pd.merge(df, df_l2y_club, left_on='L2Y Club', right_on='L2Y Club Name', how='left').drop(columns='L2Y Club Name').drop(columns='L2Y Club')
  df = pd.merge(df, df_l2y_competition, left_on='L2Y League', right_on='L2Y Competition Name', how='left').drop(columns='L2Y Competition Name').drop(columns='L2Y League')

  df = pd.merge(df, df_l3y, left_on=['L3Y Year','L3Y Club','L3Y League'], right_on=['L3Y Season','L3Y Team','L3Y League'], how='left').drop(columns=['L3Y Year','L3Y Season','L3Y Team'])
  df = pd.merge(df, df_l3y_club, left_on='L3Y Club', right_on='L3Y Club Name', how='left').drop(columns='L3Y Club Name').drop(columns='L3Y Club')
  df = pd.merge(df, df_l3y_competition, left_on='L3Y League', right_on='L3Y Competition Name', how='left').drop(columns='L3Y Competition Name').drop(columns='L3Y League')

  df = df.drop(columns=['LY GD','L2Y GD','L3Y GD'])

  df['CA'] = df['CA'] / 200
  df['PA'] = df['PA'] / 200

  df['LY App'] = df['LY App'] / df['LY Pld']
  df['LY Goal'] = df['LY Goal'] / df['LY Pld']
  df['LY W'] = df['LY W'] / df['LY Pld']
  df['LY D'] = df['LY D'] / df['LY Pld']
  df['LY L'] = df['LY L'] / df['LY Pld']
  df['LY GF'] = df['LY GF'] / df['LY Pld']
  df['LY GA'] = df['LY GA'] / df['LY Pld']
  df['LY Pts'] = df['LY Pts'] / df['LY Pld']

  df['L2Y App'] = df['L2Y App'] / df['L2Y Pld']
  df['L2Y Goal'] = df['L2Y Goal'] / df['L2Y Pld']
  df['L2Y W'] = df['L2Y W'] / df['L2Y Pld']
  df['L2Y D'] = df['L2Y D'] / df['L2Y Pld']
  df['L2Y L'] = df['L2Y L'] / df['L2Y Pld']
  df['L2Y GF'] = df['L2Y GF'] / df['L2Y Pld']
  df['L2Y GA'] = df['L2Y GA'] / df['L2Y Pld']
  df['L2Y Pts'] = df['L2Y Pts'] / df['L2Y Pld']

  df['L3Y App'] = df['L3Y App'] / df['L3Y Pld']
  df['L3Y Goal'] = df['L3Y Goal'] / df['L3Y Pld']
  df['L3Y W'] = df['L3Y W'] / df['L3Y Pld']
  df['L3Y D'] = df['L3Y D'] / df['L3Y Pld']
  df['L3Y L'] = df['L3Y L'] / df['L3Y Pld']
  df['L3Y GF'] = df['L3Y GF'] / df['L3Y Pld']
  df['L3Y GA'] = df['L3Y GA'] / df['L3Y Pld']
  df['L3Y Pts'] = df['L3Y Pts'] / df['L3Y Pld']

  df['Nation Rep'] = df['Nation Rep'] / 10000
  df['Club Rep'] = df['Club Rep'] / 10000
  df['Competition Rep'] = df['Competition Rep'] / 200

  df['LY Club Rep'] = df['LY Club Rep'] / 10000
  df['LY Competition Rep'] = df['LY Competition Rep'] / 200

  df['L2Y Club Rep'] = df['L2Y Club Rep'] / 10000
  df['L2Y Competition Rep'] = df['L2Y Competition Rep'] / 200

  df['L3Y Club Rep'] = df['L3Y Club Rep'] / 10000
  df['L3Y Competition Rep'] = df['L3Y Competition Rep'] / 200

  df['Inter. Goals'] = df.apply(lambda row: reformat_internal_goals(row['Inter. Apps'], row['Inter. Goals']), axis=1)
  df['Inter. Apps'] = df['Inter. Apps'] / 255

  df['DOB_FromNow'] = df['DOB_FromNow'] / max_diff.total_seconds()

  df = df.drop(columns=['LY Pld','L2Y Pld','L3Y Pld'])
  
  df = df.set_index('Name')

  return df

def get_X_and_y(df):
  X = df[:].copy().drop(columns=['CA','PA'])
  y = df['CA'].copy()

  return X, y

In [0]:
df_data = load_data()

In [0]:
df_data.head()

,Inter. Apps,Inter. Goals,GK,DF,DM/WB,MF,AMF,FW,LY App,LY Goal,L2Y App,L2Y Goal,L3Y App,L3Y Goal,CA,PA,DOB_FromNow,Nation Rep,Club Rep,Competition Rep,LY W,LY D,LY L,LY GF,LY GA,LY Pts,LY Club Rep,LY Competition Rep,L2Y W,L2Y D,L2Y L,L2Y GF,L2Y GA,L2Y Pts,L2Y Club Rep,L2Y Competition Rep,L3Y W,L3Y D,L3Y L,L3Y GF,L3Y GA,L3Y Pts,L3Y Club Rep,L3Y Competition Rep
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aaron Wan-Bissaka,0.000000,0.000000,0,1,1,0,0,0,0.921053,0.000000,0.184211,0.000000,0.000000,0.000000,0.720,0.810,0.446755,0.705,0.855,0.870,0.368421,0.184211,0.447368,1.342105,1.394737,1.289474,0.660,0.870,0.289474,0.289474,0.421053,1.184211,1.447368,1.157895,0.6600,0.870,0.315789,0.131579,0.552632,1.315789,1.657895,1.078947,0.6600,0.870
Abdu Conté,0.000000,0.000000,0,1,0,0,0,0,0.000000,0.000000,0.631579,0.000000,0.166667,0.000000,0.575,0.640,0.440349,0.800,0.555,0.760,0.676471,0.147059,0.176471,2.117647,0.970588,2.176471,0.700,0.760,0.289474,0.236842,0.473684,1.210526,1.710526,1.105263,0.0001,0.505,0.357143,0.238095,0.404762,1.523810,1.476190,1.309524,0.0001,0.505
Alisson,0.164706,0.000000,1,0,0,0,0,0,1.000000,0.000000,0.973684,0.000000,0.000000,0.000000,0.870,0.900,0.548870,0.915,0.900,0.870,0.789474,0.184211,0.026316,2.342105,0.578947,2.552632,0.900,0.870,0.605263,0.210526,0.184211,1.605263,0.736842,2.026316,0.8100,0.845,0.736842,0.078947,0.184211,2.368421,1.000000,2.289474,0.8100,0.845
Andrew Robertson,0.117647,0.100000,0,1,1,0,0,0,0.947368,0.000000,0.578947,0.026316,0.868421,0.026316,0.785,0.825,0.520369,0.550,0.900,0.870,0.789474,0.184211,0.026316,2.342105,0.578947,2.552632,0.900,0.870,0.552632,0.315789,0.131579,2.210526,1.000000,1.973684,0.9000,0.870,0.236842,0.184211,0.578947,0.973684,2.105263,0.894737,0.5700,0.870
Antoine Griezmann,0.239216,0.393443,0,0,0,0,1,1,0.973684,0.394737,0.842105,0.500000,0.947368,0.421053,0.900,0.925,0.579325,0.875,0.920,0.915,0.578947,0.263158,0.157895,1.447368,0.763158,2.000000,0.875,0.915,0.605263,0.263158,0.131579,1.526316,0.578947,2.078947,0.8750,0.915,0.605263,0.236842,0.157895,1.842105,0.710526,2.052632,0.8750,0.915


In [0]:
df_data = df_data.fillna(0)

In [0]:
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# scaler = StandardScaler()
# scaler.fit(df_data)
# df_data_std = pd.DataFrame(scaler.transform(df_data), columns=df_data.columns, index=df_data.index)

In [0]:
X, y = get_X_and_y(df_data)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn import feature_selection
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

pipe = Pipeline([('pca', PCA(random_state=2020)), ('svr', SVR(gamma='scale'))])

In [0]:
param_grid = {
  'svr__kernel': ['linear','rbf', 'sigmoid'],
  'pca__n_components': [20, 30, 35],
  'svr__C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50],
  'svr__epsilon': [0.005 ,0.01, 0.05, 0.1, 0.5, 1, 10],
}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=2020,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('svr',
                                        SVR(C=1.0, cache_size=200, coef0=0.0,
                                            degree=3, epsilon=0.1,
                                            gamma='scale', kernel='rbf',
                                            max_iter=-1, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'pca__n_

In [0]:
print("\nGrid-Search")
print("Best parameters:", grid.best_params_)
print("Best cross-validation score: {:.3f}".format(grid.best_score_))


Grid-Search
Best parameters: {'pca__n_components': 20, 'svr__C': 50, 'svr__epsilon': 0.01, 'svr__kernel': 'linear'}
Best cross-validation score: 0.895


In [0]:
scores = grid.score(X_test, y_test)
print("Test set score: {:.2f}".format(scores))

Test set score: 0.89


In [0]:
predict_result = grid.predict(X_test)

In [0]:
df_predict = X_test[:].copy()
df_predict['predict CA'] = predict_result * 200
df_predict['predict CA']

Name
Ryan Haynes                92.257990
James Collins             113.651794
Tom Eaves                 107.558391
Marcos Rojo               152.897958
Rhys Evitt-Healey          88.648206
Adam Roscrow               97.979145
Trent Alexander-Arnold    155.397460
John Marquis              110.075015
David Marshall            129.337013
Herbie Kane               133.901547
Steven May                110.389458
Aaron Ramsdale            120.926448
Jota                      104.881446
Yapp Hung Fai              80.660022
John Marquis              110.075015
Daniel James              130.783388
Mason Bennett             117.446026
Fábio Silva               127.192468
Ryan Fredericks           128.340031
John Marquis              110.075015
Harry Maguire             142.761612
Hélio                      78.116641
Lukasz Fabianski          147.498742
Kenny Steppe              139.552942
Liam Shephard             103.757988
Diego Eli                  80.770178
David De Gea              163.840

In [0]:
from sklearn.externals import joblib

model_filename = 'model.joblib'
joblib.dump(grid, model_filename)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['model.joblib']